# Setting up Paths

In [1]:
#path to the data
pathCOMPASOutput = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'
#path to compasrepo which contains ther popsynth folder
pathRepo = '/home/cneijssel/Documents/COMPASpop'
#path to Bootstrap
pathBootstrap = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'


# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import os
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append(pathRepo + '/popsynth/Papers/NeijsselEtAL/CosmicIntegration/Scripts')


# Custom Imports from COMPASrepo

In [3]:
import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
import ClassBootStrap         as CB #pre sampled from initial seeds per metallicity the reader is there too
import coencodeVarious        as CV
import ClassBayes             as CL  #Used to create KDECOMPAS/Posterior/Likelyhood 
import ClassEvents            as CE  #Where I store the GW-events observed


# General Idea

This combines the rates and likelihood results for the main table

In [4]:
#         [0,1,2]   [3,4,5]   [6,7,8]   [9,10,11]  [12,13,14]
#L columns   McO1-+ RO1-+     McO2-+     RO2-+     Ltot-+
#            [0,1,2]    [3,4,5]   [6,7,8]      [9,10,11]   [12,13,14]  [15,16,17]
#R columns = BBH Obs &  BBH z=0  & BHNS Obs  & BHNS z=0 &  BNS Obs & BNS z=0 
oL  = np.loadtxt('LikelihoodsOptimistic.txt')
pL  = np.loadtxt('LikelihoodsPessimistic.txt')
oR  = np.loadtxt('RatesOptimistic.txt')
pR  = np.loadtxt('RatesPessimistic.txt')


#the files are ordered with the same variations
#combine them in new results array for easier printing

#columns = BBH BHNS BNS LMc LR Ltot  (no error bars so 6 columns 18 variations)
o        = np.zeros(shape=(18,6))
o[:,0]   = oR[:,0]  #BBH
o[:,1]   = oR[:,6]  #BHNS
o[:,2]   = oR[:,12] #BNS
o[:,3]   = np.add(oL[:,0], oL[:,6]) #Lmc tot
o[:,4]   = np.add(oL[:,3], oL[:,9]) #LR  tot
o[:,5]   = oL[:,12]

#print np.add(o[:,3], o[:,4])
#print o[:,5]



#columns = BBH BHNS BNS LMc LR Ltot  (no error bars so 6 columns 18 variations)
p        = np.zeros(shape=(18,6))
p[:,0]   = pR[:,0]  #BBH
p[:,1]   = pR[:,6]  #BHNS
p[:,2]   = pR[:,12] #BNS
p[:,3]   = np.add(pL[:,0], pL[:,6]) #Lmc tot
p[:,4]   = np.add(pL[:,3], pL[:,9]) #LR  tot
p[:,5]   = pL[:,12]

print p[:,3]
print p[:,5]

[-33.90132988 -32.42209207 -32.47923602 -32.23573001 -32.60862832
 -32.77281298 -32.30432394 -32.68407745 -32.87289791 -33.82371569
 -32.80701404 -32.64662089 -32.43929589 -32.98485115 -33.08891159
 -32.45776248 -33.20094135 -33.2234479 ]
[ -34.86818006  -41.28140683  -44.38259041  -67.08519109 -103.20872203
 -113.00032587  -33.37576681  -45.61126481  -50.49214084  -35.1320506
  -59.94767911  -60.54303704  -96.54754265 -144.90105212 -154.87401259
  -40.63307633  -71.68295846  -76.32728187]


In [5]:



starter     = ['Madau et al. & Ma et al. (2004) & 1 &',' & & 2 &',' & & 3 &',
               ' & Langer et al.  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               'Strolger et al. & Ma et al. (2004) & 1 &',\
               ' & &2 &',\
               ' & &3 &',\
               ' & Langer et al. &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
              ]
print r'\begin{table*}'
print r'\begin{tabular}{|l |l | l| llllll}'
print r'\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{3}{|c|}{ Rates} & \multicolumn{3}{|c|}{ Likelihoods} \\'
print r'SFR & GSMF & MZ & $\rm BBH $ &   $\rm BHNS $  &$\rm BNS$ &   $\rm \mathcal{L}_{M_{chirp}}$ & $\rm \mathcal{L}_{R}$ & $\rm \mathcal{L}_{tot}$\\ \hline \hline'


for nrV in range(len(p[:,0])):
    string = starter[nrV]
    row    = p[nrV]
    row = np.round(row, 2)
    string+= r'%s & %s & %s & %s & %s & %s \\'\
    %(row[0],row[1], row[2],row[3],row[4],row[5])
        
    if nrV in [2,5,11,14]:
        string += '\cline{2-9}'
    if nrV == 8:
        string += '\hline'
    print string


print r'\end{tabular}'
print r'\end{table*}'

\begin{table*}
\begin{tabular}{|l |l | l| llllll}
\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{3}{|c|}{ Rates} & \multicolumn{3}{|c|}{ Likelihoods} \\
SFR & GSMF & MZ & $\rm BBH $ &   $\rm BHNS $  &$\rm BNS$ &   $\rm \mathcal{L}_{M_{chirp}}$ & $\rm \mathcal{L}_{R}$ & $\rm \mathcal{L}_{tot}$\\ \hline \hline
Madau et al. & Ma et al. (2004) & 1 &18.43 & 0.4 & 0.11 & -33.9 & -0.97 & -34.87 \\
 & & 2 &94.35 & 0.51 & 0.12 & -32.42 & -8.86 & -41.28 \\
 & & 3 &113.92 & 0.52 & 0.13 & -32.48 & -11.9 & -44.38 \\\cline{2-9}
 & Langer et al.  &1 &247.22 & 1.28 & 0.22 & -32.24 & -34.85 & -67.09 \\
 &  &2 &441.08 & 1.19 & 0.22 & -32.61 & -70.6 & -103.21 \\
 &  &3 &492.27 & 1.25 & 0.23 & -32.77 & -80.23 & -113.0 \\\cline{2-9}
 & Langer et al., offset  &1 &28.72 & 0.23 & 0.09 & -32.3 & -1.07 & -33.38 \\
 &  &2 &120.3 & 0.35 & 0.11 & -32.68 & -12.93 & -45.61 \\
 &  &3 &148.74 & 0.35 & 0.11 & -32.87 & -17.62 & -50.49 \\\hline
Strolger et al. & Ma et al. (2004) & 1 &32.93 & 0.52 & 0.12 & -33.82 & 